In [ ]:
import tensorflow.compat.v1 as tf1
# import tensorflow as tf2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1
session = tf1.Session(config=config)

In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras import backend as K  # 추가된 부분
from tensorflow.keras import layers  # Conv2D, MaxPooling2D 등 일부를 layers에서 가져올 수도 있습니다

def open_image(image_dir, image_file):
    """
    주어진 디렉터리와 파일명을 사용하여 이미지를 읽고, Grayscale 형식으로 변환합니다.
    
    매개변수:
    - image_dir (str): 이미지 파일이 저장된 디렉터리 경로.
    - image_file (str): 읽을 이미지 파일 이름.
    
    반환값:
    - Grayscale로 변환된 이미지 (numpy.ndarray).
    """
    image_path = os.path.join(image_dir, image_file)  # 디렉터리와 파일 이름을 결합하여 이미지 경로 생성
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY)  # 이미지 읽고 Grayscale로 변환


def visualize_width_image(images):
    """
    여러 이미지를 가로로 출력합니다.
    
    매개변수:
    - images (list): 시각화할 이미지의 리스트.
    """
    plt.figure(figsize=(5 * len(images), 5))  # 이미지 갯수에 따라 가로 크기 조정
    for idx, image in enumerate(images):
        plt.subplot(1, len(images), idx + 1)  # 가로로 이미지를 나란히 출력
        plt.imshow(image, cmap='gray')  # Grayscale 형식으로 이미지 출력
        plt.axis('off')  # 축 비활성화
    plt.show()  # 시각화 출력


def visualize_hight_image(images):
    """
    여러 이미지를 세로로 출력합니다.
    
    매개변수:
    - images (list): 시각화할 이미지의 리스트.
    """
    plt.figure(figsize=(5, 5 * len(images)))  # 이미지 갯수에 따라 세로 크기 조정
    for idx, image in enumerate(images):
        plt.subplot(len(images), 1, idx + 1)  # 세로로 이미지를 나란히 출력
        plt.imshow(image, cmap='gray')  # Grayscale 형식으로 이미지 출력
        plt.axis('off')  # 축 비활성화
    plt.show()  # 시각화 출력


In [ ]:
def plot_accuracy_loss(history):
    """
    훈련 및 검증 정확도와 손실을 시각화합니다.
    
    매개변수:
    - history: 정확도 및 손실 값을 포함하는 훈련 기록 객체.
    """
    plt.figure(figsize=(10, 4))

    # 정확도 그래프
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Model Accuracy')

    # 손실 그래프
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('에포크(Epoch)')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Model Loss')

    plt.tight_layout()
    plt.show()

def calculate_f1_score(model, x_val, y_val, batch_size=4):
    """
    F1 점수를 계산하고 출력합니다.
    
    매개변수:
    - model: 검증 데이터 세트에서 예측을 생성할 훈련된 모델.
    - x_val: 검증 데이터 입력값.
    - y_val: 검증 데이터 레이블.
    - batch_size: 예측을 위한 배치 크기(기본값: 4).
    
    반환값:
    - y_pred: 예측된 레이블.
    - y_true: 실제 레이블.
    """
    y_pred = np.argmax(model.predict(x_val, batch_size=batch_size), axis=-1)
    y_true = np.argmax(y_val, axis=-1)

    f1 = f1_score(y_true.flatten(), y_pred.flatten(), average='weighted')
    print(f"F1 점수: {f1:.2f}")

    return y_true, y_pred

def plot_confusion_matrix_with_metrics(y_true, y_pred):
    """
    혼동 행렬을 시각화하고 정밀도, 재현율, F1 점수를 출력합니다.
    
    매개변수:
    - y_true: 실제 레이블.
    - y_pred: 예측된 레이블.
    """
    # Flatten 데이터
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    # 혼동 행렬 계산
    cm = confusion_matrix(y_true, y_pred)
    
    # 메트릭 계산 (zero_division=0 설정)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    
    # 메트릭 출력
    print("=== 성능 지표 ===")
    print(f"정밀도(Precision): {precision:.2f}")
    print(f"재현율(Recall): {recall:.2f}")
    
    # 혼동 행렬 시각화
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues', values_format='d')
    plt.title('Confusion Matrix')
    plt.show()
    
def plot_true_vs_pred_images(model, x_val, y_val, num_images=5, batch_size=4):
    """
    실제 이미지, 라벨 이미지 및 예측된 이미지를 나란히 시각화합니다.
    
    매개변수:
    - model: 검증 데이터 세트에서 예측을 생성할 훈련된 모델.
    - x_val: 검증 데이터 입력값.
    - y_val: 검증 데이터 레이블.
    - num_images: 시각화할 이미지 수 (기본값: 5).
    - batch_size: 예측을 위한 배치 크기(기본값: 4).
    """
    # 예측 생성
    y_pred = model.predict(x_val, batch_size=batch_size)
    
    plt.figure(figsize=(25, 3 * num_images))
    for i in range(num_images):
        # 실제 입력 이미지
        plt.subplot(num_images, 5, 5 * i + 1)
        plt.imshow(x_val[i].squeeze(), cmap='gray')
        plt.title('Original Image')
        plt.axis('off')
        
        # 실제 라벨 이미지 (첫 번째 채널 선택 - 전경)
        plt.subplot(num_images, 5, 5 * i + 2)
        plt.imshow(y_val[i][..., 1].squeeze(), cmap='gray')  # 전경 채널 시각화
        plt.title('Label Image (Foreground)')
        plt.axis('off')
        
        # 예측된 이미지 (전체 클래스 비교 - argmax로 클래스 라벨 선택)
        plt.subplot(num_images, 5, 5 * i + 3)
        plt.imshow(np.argmax(y_val[i], axis=-1).squeeze(), cmap='gray')
        plt.title('Original Label (Argmax)')
        plt.axis('off')
        
        # 예측된 이미지 (첫 번째 채널 선택 - 전경)
        plt.subplot(num_images, 5, 5 * i + 4)
        plt.imshow(y_pred[i][..., 1].squeeze(), cmap='gray')  # 전경 채널 시각화
        plt.title('Predicted Image (Foreground)')
        plt.axis('off')
        
        # 예측된 이미지 (전체 클래스 비교 - argmax로 클래스 라벨 선택)
        plt.subplot(num_images, 5, 5 * i + 5)
        plt.imshow(np.argmax(y_pred[i], axis=-1).squeeze(), cmap='gray')
        plt.title('Predicted Label (Argmax)')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
def main_display_result(history, model, x_val, y_val, batch_size=4, num_images=5):
    """
    전체 워크플로우 실행: 정확도/손실 그래프 시각화, F1 점수 계산, 혼동 행렬 및 메트릭 출력, 실제/예측 이미지 비교.
    
    매개변수:
    - history: 정확도 및 손실 값을 포함하는 훈련 기록 객체.
    - model: 검증 데이터 세트에서 예측을 생성할 훈련된 모델.
    - x_val: 검증 데이터 입력값.
    - y_val: 검증 데이터 레이블.
    - batch_size: 예측을 위한 배치 크기(기본값: 4).
    - num_images: 시각화할 이미지 수 (기본값: 5).
    """
    # 훈련 및 검증 정확도/손실 그래프 시각화
    plot_accuracy_loss(history)
    
    # F1 점수 계산 및 예측값 반환
    y_true, y_pred = calculate_f1_score(model, x_val, y_val, batch_size=batch_size)
    
    # 혼동 행렬 및 메트릭 출력
    plot_confusion_matrix_with_metrics(y_true, y_pred)
    
    # 실제 이미지와 예측된 이미지 비교 시각화
    plot_true_vs_pred_images(model, x_val, y_val, num_images=num_images, batch_size=batch_size)

In [ ]:
def open_json(json_path):
    """
    JSON 데이터를 불러옵니다.
    
    매개변수:
    - json_path (str): JSON 파일의 경로.
    
    반환값:
    - json_data (dict): JSON 데이터로 파싱된 Python 딕셔너리 객체.
    """
    with open(json_path, "r") as f:
        json_data = json.load(f)  # JSON 파일 읽기 및 파싱
    
    return json_data


def get_polygon_from_json(json_data, image_filename):
    """
    JSON 데이터에서 특정 이미지에 대한 다각형(Polygon) 데이터를 추출합니다.
    
    매개변수:
    - json_data (dict): JSON에서 로드된 데이터.
    - image_filename (str): 다각형 정보를 추출할 이미지 파일 이름.
    
    반환값:
    - polygons (list): 다각형의 x, y 좌표 리스트 [(x_coords, y_coords), ...].
    """
    regions = json_data[image_filename]["regions"]  # 이미지와 관련된 영역 정보 가져오기
    polygons = []  # 다각형 정보를 저장할 리스트
    
    for region_id, region_info in regions.items():  # 영역 정보 순회
        shape_attributes = region_info["shape_attributes"]  # 모양 정보 추출
        if shape_attributes["name"] == "polygon":  # 다각형인지 확인
            x_coords = shape_attributes["all_points_x"]  # x 좌표
            y_coords = shape_attributes["all_points_y"]  # y 좌표
            polygons.append((x_coords, y_coords))  # 다각형 정보 추가
    
    return polygons


def creat_mask(image):
    """
    주어진 이미지 크기에 따라 빈 마스크 이미지를 생성합니다.
    
    매개변수:
    - image (numpy.ndarray): 원본 이미지.
    
    반환값:
    - image (numpy.ndarray): 동일한 크기의 빈 마스크 이미지 (값이 0으로 초기화된 배열).
    """
    image = np.zeros(image.shape)  # 이미지 크기와 동일한 배열 생성 (0으로 초기화)
    
    return image


def fill_polygons(image, polygons):
    """
    빈 마스크 이미지에 다각형 영역을 채웁니다.
    
    매개변수:
    - image (numpy.ndarray): 빈 마스크 이미지.
    - polygons (list): 다각형의 x, y 좌표 리스트 [(x_coords, y_coords), ...].
    
    반환값:
    - image (numpy.ndarray): 다각형이 채워진 마스크 이미지.
    """
    image = image.copy()  # 원본 이미지를 복사하여 수정
    for x_coords, y_coords in polygons:  # 다각형 리스트 순회
        pts = np.array([list(zip(x_coords, y_coords))], dtype=np.int32)  # 좌표를 다각형 형태로 변환
        cv2.fillPoly(image, pts, color=255)  # 다각형 내부를 흰색(255)으로 채움
    
    return image


def crate_mask_image(image_dir, json_path, image_file):
    """
    이미지와 JSON 데이터를 기반으로 다각형이 채워진 마스크 이미지를 생성합니다.
    
    매개변수:
    - image_dir (str): 원본 이미지 파일이 저장된 디렉터리 경로.
    - json_path (str): 다각형 정보를 포함하는 JSON 파일의 경로.
    - image_file (str): 처리할 이미지 파일 이름.
    
    반환값:
    - mask_image (numpy.ndarray): 생성된 마스크 이미지.
    """
    json_data = open_json(json_path)  # JSON 데이터 로드
    image = open_image(image_dir, image_file)  # 원본 이미지 로드
    
    mask_image = creat_mask(image)  # 빈 마스크 이미지 생성
                       
    return fill_polygons(mask_image, get_polygon_from_json(json_data, image_file))  # 다각형을 마스크에 채움

In [ ]:
def image_resize(image, shape):
    """
    입력된 이미지의 크기를 지정된 shape로 조정합니다.
    
    매개변수:
    - image (numpy.ndarray): 크기 조정할 이미지.
    - shape (tuple): 조정할 크기 (높이, 너비).
    
    반환값:
    - resized_image (numpy.ndarray): 조정된 크기의 이미지.
    
    주의: 항상 채널 차원이 추가된 이미지를 반환합니다.
    """
    resized_image = cv2.resize(image, shape)  # 입력 이미지의 크기를 지정된 shape로 조정
    return resized_image


def expand_dims(image):
    """
    입력된 이미지를 채널 차원(axis=-1)까지 확장합니다.
    
    매개변수:
    - image (numpy.ndarray): 채널 차원을 확장할 이미지.
    
    반환값:
    - expanded_image (numpy.ndarray): 채널 차원이 추가된 이미지.
    """
    expanded_image = np.expand_dims(image, axis=-1)  # 이미지를 채널 차원까지 확장
    return expanded_image

def prepare_and_count_labels(y_train, y_val):
    """
    주어진 레이블 데이터(y_train, y_val)에 대해 0과 1로 이진화하고, 각 레이블의 빈도를 출력합니다.
    
    매개변수:
    - y_train (numpy.ndarray): 학습 데이터 레이블.
    - y_val (numpy.ndarray): 검증 데이터 레이블.
    
    반환값:
    - y_train.shape (tuple): 학습 레이블 크기.
    - y_val.shape (tuple): 검증 레이블 크기.
    """
    # 이진화: 0보다 큰 값을 1로, 0은 그대로 0으로 설정
    y_train = np.where(y_train > 0, 1, 0)
    y_val = np.where(y_val > 0, 1, 0)
    
    # y_train의 값별 빈도 계산
    unique, counts = np.unique(y_train, return_counts=True)
    value_counts = dict(zip(unique, counts))
    
    # 빈도 출력
    print("학습 데이터셋 레이블 빈도:")
    for value, count in value_counts.items():
        print(f"값: {value}, 빈도: {count}")
    
    # y_val의 값별 빈도 계산
    unique, counts = np.unique(y_val, return_counts=True)
    value_counts = dict(zip(unique, counts))
    
    # 빈도 출력
    print("검증 데이터셋 레이블 빈도:")
    for value, count in value_counts.items():
        print(f"값: {value}, 빈도: {count}")
    
    return y_train, y_val

def normalize_image(image):
    """
    이미지를 정규화하여 0에서 1 사이의 값으로 변환합니다.
    
    매개변수:
    - image (numpy.ndarray): 정규화할 이미지 배열.
    
    반환값:
    - normalized_image (numpy.ndarray): 정규화된 이미지 배열.
    """
    normalized_image = image.astype(np.float32) / 255.0
    return normalized_image

def add_background_channel(mask):
    """
    마스크에서 배경 채널을 추가합니다.
    
    매개변수:
    - mask (numpy.ndarray): 단일 채널 마스크 (0: 배경, 1: 객체).
    
    반환값:
    - multi_channel_mask (numpy.ndarray): 2채널 마스크 (채널 0: 배경, 채널 1: 객체).
    """
    background = 1 - mask  # 배경 채널은 객체 채널의 반전값
    multi_channel_mask = np.stack([background, mask], axis=-1)  # 채널 축으로 쌓음
    return multi_channel_mask

def regularize_mask(mask):
    """
    생성된 마스크 이미지를 정규화하여 잡음을 제거하고 품질을 개선합니다.
    
    매개변수:
    - mask (numpy.ndarray): 생성된 마스크 이미지.
    
    반환값:
    - regularized_mask (numpy.ndarray): 정규화된 마스크 이미지.
    """
    # Gaussian Blur로 잡음 제거
    blurred = cv2.GaussianBlur(mask, (5, 5), 0)
    
    # Binary Threshold 적용
    _, binary = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)
    
    # Morphological Transformations (열림 연산)으로 작은 잡음 제거
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    
    return cleaned

def prepare_dataset(image_dir, json_path, image_size=(512, 512), test_size=0.2, random_state=42):
    """
    주어진 이미지 디렉토리와 JSON 파일로부터 데이터를 준비하고 학습/검증 데이터셋을 반환합니다.
    데이터 내 0과 1 이외의 값이 포함된 경우 이진화 처리 및 빈도 계산을 수행합니다.

    매개변수:
    - image_dir (str): 이미지가 저장된 디렉터리 경로.
    - json_path (str): 폴리곤 마스크 정보가 포함된 JSON 파일 경로.
    - image_size (tuple): 이미지 크기 조정할 크기 (기본값: (512, 512)).
    - test_size (float): 테스트 데이터 비율 (기본값: 0.2).
    - random_state (int): 랜덤 상태 (기본값: 42).

    반환값:
    - x_train (numpy.ndarray): 학습 이미지.
    - x_val (numpy.ndarray): 검증 이미지.
    - y_train (numpy.ndarray): 학습 이미지 마스크 (2채널).
    - y_val (numpy.ndarray): 검증 이미지 마스크 (2채널).
    """
    image = []
    image_mask = []

    for image_file in os.listdir(image_dir):
        if image_file.endswith('.jpg'):  # 이미지 파일만 처리
            # 이미지와 마스크 데이터를 크기 조정 및 채널 차원 확장
            img = np.expand_dims(image_resize(open_image(image_dir, image_file), image_size), axis=-1)
            img = normalize_image(img)  # 이미지 정규화
            image.append(img)

            # 마스크 생성 및 크기 조정
            mask = image_resize(crate_mask_image(image_dir, json_path, image_file), image_size)
            
            # 정규화 과정 적용
            mask = regularize_mask(mask)
            
            # 채널 차원 추가 및 이진화
            mask = np.expand_dims(mask, axis=-1)
            mask = np.where(mask > 0, 1, 0)  # 이진화
            
            # 배경 채널 추가
            mask = add_background_channel(mask)
            image_mask.append(mask)

    # NumPy 배열로 변환
    image = np.array(image)
    image_mask = np.array(image_mask)

    # 학습용과 검증용 데이터로 분할
    x_train, x_val, y_train, y_val = train_test_split(image, image_mask, test_size=test_size, random_state=random_state)

    # y_train과 y_val에서 0과 1 외의 값이 포함되어 있는지 확인
    unique_train = np.unique(y_train)
    unique_val = np.unique(y_val)

    if len(set(unique_train) - {0, 1}) > 0 or len(set(unique_val) - {0, 1}) > 0:
        print("경고: 레이블에 0과 1 이외의 값이 포함되어 있습니다. 레이블 준비를 수행합니다.")
        y_train, y_val = prepare_and_count_labels(y_train, y_val)
    else:
        print("레이블이 이미 이진화되어 있습니다 (0과 1만 포함).")

    # 크기 확인 및 불일치 시 처리
    x_train = np.squeeze(x_train)
    x_val = np.squeeze(x_val)
    y_train = np.squeeze(y_train)
    y_val = np.squeeze(y_val)

    # 크기 출력
    print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
    print(f"x_val shape: {x_val.shape}, y_val shape: {y_val.shape}")

    return x_train, x_val, y_train, y_val

In [ ]:
image_dir = 'FracAtlas/images/Fractured'
json_path = 'FracAtlas/Annotations/VGG JSON/VGG_fracture_masks.json'

x_train, x_val, y_train, y_val = prepare_dataset(image_dir, json_path, image_size=(512, 512), test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import EarlyStopping

# U-Net 모델 정의
def unet_model(input_shape, num_classes=2):
    """
    U-Net 모델 정의 함수.
    
    매개변수:
    - input_shape: 입력 이미지 형태 (높이, 너비, 채널).
    - num_classes: 출력 채널 수. 기본값은 2 (배경과 객체).

    반환값:
    - Keras 모델 객체.
    """
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
    u2 = concatenate([u2, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u1 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u1 = concatenate([u1, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c5)  # num_classes에 따라 출력 채널 설정
    model = Model(inputs=[inputs], outputs=[outputs])

    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# EarlyStopping 콜백 정의 (IoU를 기준으로 모니터링)
def early_stopping_iou_callback(weight=1.0):
    early_stopping = EarlyStopping(
        monitor='val_weighted_iou',   # IoU를 모니터링
        patience=3,                  # 개선이 없을 때 기다릴 에포크 수
        restore_best_weights=True,   # 가장 좋은 모델의 가중치를 복원
        mode='max',                  # IoU 값이 최대일 때 성능이 좋은 것으로 간주
    )
    return early_stopping

# Focal Loss 정의 (레이블 불균형 대응)
def focal_loss(alpha=0.25, gamma=2.0):
    def loss_fn(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1. - tf.keras.backend.epsilon())
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.reduce_mean(alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt))
    return loss_fn

# Jaccard Index (IoU) 계산 함수 정의
def iou(y_true, y_pred, smooth=1e-4):
    y_pred = K.argmax(y_pred, axis=-1)
    y_true = K.argmax(y_true, axis=-1)
    y_true = K.cast(y_true, dtype='float32')
    y_pred = K.cast(y_pred, dtype='float32')
    intersection = K.sum(K.abs(y_true * y_pred), axis=(1, 2))
    union = K.sum(y_true, axis=(1, 2)) + K.sum(y_pred, axis=(1, 2)) - intersection
    iou_score = (intersection + smooth) / (union + smooth)
    return K.mean(iou_score)

# 모델 컴파일 (IoU 및 Focal Loss)
def compile_model_with_focal_loss(model, weight=1.0):
    def weighted_iou(y_true, y_pred):
        iou_score = iou(y_true, y_pred)
        return iou_score * weight  # IoU에 가중치를 곱함

    model.compile(
        optimizer=Adam(),
        loss=focal_loss(alpha=0.25, gamma=2.0),  # Focal Loss 사용
        metrics=['accuracy', weighted_iou]
    )
    return model

# 데이터 증강 적용 및 입력 데이터 차원 조정
def augment_data(x_train, y_train, batch_size=4):
    """
    데이터 증강을 수행하고 입력 데이터 차원을 확인 및 수정합니다.
    """
    # 입력 데이터 차원 확장
    if x_train.ndim == 3:
        x_train = np.expand_dims(x_train, axis=-1)  # (batch_size, height, width) -> (batch_size, height, width, 1)
    if y_train.ndim == 3:
        y_train = np.expand_dims(y_train, axis=-1)  # 동일하게 채널 차원 추가

    # 데이터 증강 생성기
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # 데이터 제너레이터 생성
    generator = datagen.flow(x_train, y_train, batch_size=batch_size)
    return generator

# 데이터 증강 적용 및 입력 데이터 차원 조정
    """
    데이터 증강을 수행합니다.
    
    매개변수:
    - x_train (numpy.ndarray): 학습 이미지 데이터.
    - y_train (numpy.ndarray): 학습 레이블 데이터.
    
    반환값:
    - generator: 증강된 데이터를 생성하는 데이터 제너레이터.
    """
    # 입력 데이터 차원 확장
    if x_train.ndim == 3:
        x_train = np.expand_dims(x_train, axis=-1)
    if y_train.ndim == 3:
        y_train = np.expand_dims(y_train, axis=-1)

    # 데이터 증강 생성기
    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # 데이터 제너레이터 생성
    generator = datagen.flow(x_train, y_train, batch_size=batch_size)
    return generator

# 모델 학습 함수 (Focal Loss 및 IoU 적용)
def train_unet_model_with_focal_loss(
    x_train, y_train, x_val, y_val, batch_size=4, epochs=10, image_size=(512, 512), weight=1.0):
    """
    U-Net 모델을 학습하는 함수입니다. 데이터 증강 및 Focal Loss를 적용합니다.
    
    매개변수:
    - x_train, y_train: 학습 데이터와 레이블.
    - x_val, y_val: 검증 데이터와 레이블.
    - batch_size (int): 배치 크기.
    - epochs (int): 학습 에포크 수.
    - image_size (tuple): 이미지 크기.
    - weight (float): Focal Loss 가중치.
    """
    # U-Net 모델 초기화
    model = unet_model(input_shape=(image_size[0], image_size[1], 1))

    # 모델 컴파일 (Focal Loss 사용)
    model = compile_model_with_focal_loss(model, weight=weight)

    # EarlyStopping 콜백 설정
    early_stopping = early_stopping_iou_callback(weight=weight)

    # 데이터 증강
    train_generator = augment_data(x_train, y_train)

    # 모델 학습
    history = model.fit(
        train_generator,                # 증강된 학습 데이터 사용
        validation_data=(x_val, y_val), # 검증 데이터
        epochs=epochs,                  # 학습 에포크 수
        callbacks=[early_stopping]      # EarlyStopping 콜백
    )

    # 학습 로그 분석
    analyze_logs(history)

    return model, history

# 로그 분석 함수
def analyze_logs(history):
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    train_iou = history.history['weighted_iou']
    val_iou = history.history['val_weighted_iou']

    print("\n--- 학습 로그 분석 ---\n")
    print(f"최소 학습 손실: {min(train_loss):.4f} (에포크 {train_loss.index(min(train_loss)) + 1})")
    print(f"최소 검증 손실: {min(val_loss):.4f} (에포크 {val_loss.index(min(val_loss)) + 1})")
    print(f"최대 학습 정확도: {max(train_accuracy):.4f} (에포크 {train_accuracy.index(max(train_accuracy)) + 1})")
    print(f"최대 검증 정확도: {max(val_accuracy):.4f} (에포크 {val_accuracy.index(max(val_accuracy)) + 1})")
    print(f"최대 학습 IoU: {max(train_iou):.4f} (에포크 {train_iou.index(max(train_iou)) + 1})")
    print(f"최대 검증 IoU: {max(val_iou):.4f} (에포크 {val_iou.index(max(val_iou)) + 1})")

    if val_loss[-1] > val_loss[val_loss.index(min(val_loss))]:
        print("\n경고: 검증 손실이 다시 증가하고 있습니다. 과적합 가능성을 확인하세요.\n")

    # 손실 및 IoU 그래프 시각화
    plt.figure(figsize=(10, 6))
    plt.plot(train_loss, label='Train Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend()
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(train_iou, label='Train IoU')
    plt.plot(val_iou, label='Validation IoU')
    plt.legend()
    plt.title('IoU Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.show()

In [ ]:
# 모델 학습
model, history = train_unet_model_with_focal_loss(
    x_train=x_train,
    y_train=y_train,
    x_val=x_val,
    y_val=y_val,
    batch_size=8,
    epochs=10,
    weight=2.0
)
analyze_logs(history)
main_display_result(history, model, x_val, y_val, num_images=5, batch_size=4)

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()